In [ ]:
import json
import time
from py_web_tools import PageRipper

def gather(link):
    """Gather data
    
    input
    -------
    link to hymn site
    """
    
    data = {}
    
    page = PageRipper(link).soup
    title = page.find("div", class_="page-title")
    data["title"] = title.text.strip()
    
    meta = page.find("table", id="authority_above_fold")
    
    lyrics = page.find("div", id="at_fulltext")
    try:
        texts = lyrics.find("div", class_="authority_columns")
        paragraphs = texts.find_all("p")
        data["lyrics"] = "".join([each.text.replace(" \n", "\n\n") for each in paragraphs])
    except AttributeError:
        pass

    authors = page.find_all("div", typeof="Person")
    data["author"] = {}

    for author in authors:
        data["author"][author.find("span", property="name").text] = author.text
        # {name : about}
        
    data["source"] = link
    return data

In [ ]:
home = PageRipper("https://hymnary.org/browse/popular")
hymn_links = [each.replace('/browse/popular/', '/') for each in home.links() if "text" in each]

In [ ]:
data = []
for each in hymn_links[1:]:
    data.append(gather(each))
    time.sleep(1)

In [ ]:
with open("hymnal.json", "w+") as fh:
    json.dump(data, fh)

with open("hymnal.json", "r+") as fh:
    p = json.load(fh)

for each in p:
    for k, v in each.items():
        print("***", k)
        print(v)
    print()
    print()